# Практика
Используемые библиотеки

In [5]:
import pandas as pd
import numpy as np
import requests
import io
import re
import warnings
warnings.filterwarnings('ignore')

# Эта команда позволяет вывыводить результаты друг за другом без вызова print
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [6]:
df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';')
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


* Берем за основу что 
* "в пределах" означает от X до Y (оба показатели включительно) 
* "не более" означает от 0 до N (включительно)

In [7]:
# Ваше решение

# Обозначаем Минимумы для всех показателей
df["MinLimit"] = [
    6, # pH
    0, # Запах
    0, # Цветность
    7, # Жёсткость
    0, # Аммиак и аммоний-ион (по азоту)
    0, # Нитриты (по NO2)
    0, # Нитраты (по NO3)	
    0, # Фосфаты (P)
    0, # Хлориды (Cl)
    0, # Сульфаты (SO4)
    0, # Железо (включая хлорное железо) по Fe	
    0, # Нефть
    0, # Общая минерализация (сухой остаток)	
    0, # Окисляемость перманганатная	мг/дм3
]

# Обозначаем Максимумы для всех показателей
df["MaxLimit"] = [
    9, # pH
    3, # Запах
    30, # Цветность
    10, # Жёсткость
    1.5, # Аммиак и аммоний-ион (по азоту)
    3.3, # Нитриты (по NO2)
    45, # Нитраты (по NO3)	
    3.5, # Фосфаты (P)
    350, # Хлориды (Cl)
    500, # Сульфаты (SO4)
    0.3, # Железо (включая хлорное железо) по Fe	
    0.3, # Нефть
    1000, # Общая минерализация (сухой остаток)	
    0.5, # Окисляемость перманганатная	мг/дм3
]

# Зная заранее что в столбце Цветность пользователь может проставить не числовое значение 
# Проверяем данное поле 

if df["Результат анализа"].loc[2] == "б/цвета":
    df["Результат анализа"].loc[2] = 0

# Меняем тип столбца Результат анализа со строки на числовые значения

df["Результат анализа"] = pd.to_numeric(df["Результат анализа"], downcast="float")
#df["Результат анализа"] = df["Результат анализа"].astype(np.float32)

# Создадим новый столбец Result, где 
# 1 - соответствует нормативу
# 0 - не соответсвует нормативу

df["Result"] = df.apply(lambda x: "Соответствует" if x["Результат анализа"] >= x["MinLimit"] and x["Результат анализа"] <= x["MaxLimit"] 
                        else "Не соответствует", axis=1)

# Делаем столбец "Показатель" индексовым
df.set_index("Показатель")

,Единица измерений,Результат анализа,Норматив,MinLimit,MaxLimit,Result
Показатель,,,,,,
pH,единицы pH,8.400,в пределах 6-9,6,9.0,Соответствует
Запах,баллы,1.000,не более 2-3,0,3.0,Соответствует
Цветность,градусы,0.000,не более 30,0,30.0,Соответствует
Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,7,10.0,Соответствует
Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5",0,1.5,Соответствует
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",0,3.3,Соответствует
Нитраты (по NO3),мг/дм3,24.000,не более 45,0,45.0,Соответствует
Фосфаты (P),мг/дм3,0.360,"не более 3,5",0,3.5,Соответствует
Хлориды (Cl),мг/дм3,200.000,не более 350,0,350.0,Соответствует


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [59]:
# Ваше решение

# Заполняем ящик необходимыми фруктами
inbox = ["Апельсин"] * 5 + ["Яблоко"] * 4 

# Добавим хаоса :) 
np.random.shuffle(inbox)

import random

# По очереди рандомно вытаскиваем фрукт, параметр Replace нужен чтобы один и тот же фрукт нельзя было выбрать 2 раза (фрукт обратно в ящик мы не помещаем) 
cases = [np.random.choice(inbox, size=3, replace=False) for _ in range(100000)]

df = pd.DataFrame(cases,columns = ['FirstTry', 'SecondTry', 'ThirdTry'])

#  Ищем все случаи когда вытащили именно 3 апельсина 
df_triple = df[ (df["FirstTry"] == "Апельсин") & (df["SecondTry"] == "Апельсин") & (df["ThirdTry"] == "Апельсин")]

len_total = len(df)
len_triple = len(df_triple)

len_total
len_triple
# Делим количество случаев 3 апельсинов подряд на общее количество 
len_triple/len_total

1000000

119451

0.119451

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [60]:
# Ваше решение

# Заполняем ящик необходимыми 10 деталями, назовём их Стандарт и Дефект
inbox = ["Стандарт"] * 7 + ["Дефект"] * 3 

# Добавим хаоса :) 
np.random.shuffle(inbox)

cases = [np.random.choice(inbox, 2, replace=False) for _ in range(100000)]

df = pd.DataFrame(cases,columns = ['FirstTry', 'SecondTry'])

# Считаем что под требованием проверить ровно 2 детали - значит мастеру сначала попалась Дефектная, а при следующей попытке попалась Стандартная
df_double = df[ (df["FirstTry"] == "Дефект") & (df["SecondTry"] == "Стандарт") ]

len_total = len(df)
len_double = len(df_double)

len_total
len_double
# Делим количество случаев Дефект-Стандарт подряд на общее количество 
len_double/len_total

100000

23396

0.23396